# Distributed Profiling of Model Features with Whylogs & Fugue

It is a usual practice in the Machine Learning worls to log incoming model inference requests and outgoing predictions. These logs are then processed and aggregated later for various monitoring and drift detection purposes. However, consuming this raw data presents several pain points:
+ Machine Learning models vary widely in the number and nature of features and predictions. Some have 10 features and emit probability scores while others may have 30 features and emit a ranking. 
+ They also differ significantly in the type of features, with some having more categorical features and others having more numerical features.

It is imperative for us to devise a uniform way of processing them. We cannot have a specific monitoring logic for each model. 

In this tutorial we show how to use [Whylogs](https://whylabs.ai/whylogs) to profile the features and predictions and extract only the essential metrics from these profiles, regardless of the scale of the data.

The purposes of profiling are:
+ To normalize and compress metric data while retaining maximal information.
+ We can unify data from totally different models and process them using the same pipeline in the following step.
+ The subsequent steps will only need to handle purely numerical time series.
+ Significantly reduce the scale of the problem, so the compute can be more efficient and cost effective.

We also use the open source framework called [Fugue](https://fugue-tutorials.readthedocs.io/index.html) for its excellent abstraction layer that unifies the computing logic over Pandas, Spark, Ray or Dask.One of Fugue's most popular features is the ability to use a simple Python function call to distribute logic across many partitions of a larger dataframe. Users can provide functions with type-annotated inputs and outputs, and Fugue then converts the data based on the type annotations. This makes the custom logic independent from Fugue and Spark, removing the need for any pre-existing knowledge.

![](images/scale-up-ad.png)

In [1]:
import seaborn as sns
from matplotlib import pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# this allows plots to appear directly in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
import pandas as pd

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

In [3]:
import pandas as pd

In [4]:
demo_df = pd.read_parquet('addemo23/demo_raw_data.parquet')

## Load Model Feature and Prediction Logs

In [5]:
demo_df.head(5)

,occurred_at,model_name,version,predictions,features
0,2023-02-10 05:33:01.065,demo_model,1.0.1,59.753181,"{""feature_5"":10087.0,""feature_6"":0.5283935351,""feature_1"":0.0,""feature_3"":-16.5737745071,""featur..."
1,2023-02-10 05:05:57.562,demo_model,1.0.1,15.915874,"{""feature_5"":44.0,""feature_6"":-4.9304880877,""feature_1"":0.0,""feature_3"":-7.376023565,""feature_2""..."
2,2023-02-10 05:20:57.750,demo_model,1.0.1,25.590763,"{""feature_5"":495.0,""feature_6"":-4.5830756571,""feature_1"":1.0,""feature_3"":-7.376023565,""feature_2..."
3,2023-02-10 05:15:05.361,demo_model,1.0.1,40.450287,"{""feature_5"":3025.0,""feature_6"":-4.8169581803,""feature_1"":0.0,""feature_3"":-7.376023565,""feature_..."
4,2023-02-10 05:36:09.118,demo_model,1.0.1,24.397123,"{""feature_5"":341.0,""feature_6"":-4.2160377928,""feature_1"":1.0,""feature_3"":-7.376023565,""feature_2..."


In [6]:
demo_df.shape

(1666314, 5)

### Extract Features and Predictions from model logs

In [7]:
import json
import pandas as pd

def extract_features(df: pd.DataFrame) -> pd.DataFrame:
    json_str = "[" + (",".join(df.features)) + "]"
    feature_df = pd.DataFrame(json.loads(json_str))
    #feature_df = feature_df.reset_index(drop=True)
    return feature_df[sorted(feature_df.columns)]

In [8]:
feature_df = extract_features(demo_df)

In [9]:
feature_df.head(5)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,0.0,1.904762e+00,-16.573775,-231.864749,10087.0,0.528394
1,0.0,9.375000e-01,-7.376024,-354.924267,44.0,-4.930488
2,1.0,5.909091e+00,-7.376024,-367.064925,495.0,-4.583076
3,0.0,5.000000e+08,-7.376024,-350.691669,3025.0,-4.816958
4,1.0,3.448276e-01,-7.376024,-356.517885,341.0,-4.216038


In [10]:
feature_df.shape

(1666314, 6)

In [11]:
pd.concat([demo_df[['occurred_at', 'model_name', 'version', 'predictions']], feature_df], axis=1)

,occurred_at,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,2023-02-10 05:33:01.065,demo_model,1.0.1,59.753181,0.0,1.904762e+00,-16.573775,-231.864749,10087.0,0.528394
1,2023-02-10 05:05:57.562,demo_model,1.0.1,15.915874,0.0,9.375000e-01,-7.376024,-354.924267,44.0,-4.930488
2,2023-02-10 05:20:57.750,demo_model,1.0.1,25.590763,1.0,5.909091e+00,-7.376024,-367.064925,495.0,-4.583076
3,2023-02-10 05:15:05.361,demo_model,1.0.1,40.450287,0.0,5.000000e+08,-7.376024,-350.691669,3025.0,-4.816958
4,2023-02-10 05:36:09.118,demo_model,1.0.1,24.397123,1.0,3.448276e-01,-7.376024,-356.517885,341.0,-4.216038
...,...,...,...,...,...,...,...,...,...,...
1666309,2023-02-21 05:02:40.741,demo_model,1.0.1,21.680845,1.0,7.692308e+00,13.291135,-368.704926,561.0,-5.019625
1666310,2023-02-21 05:23:18.348,demo_model,1.0.1,65.978500,1.0,2.647059e+00,13.291135,-264.067607,10637.0,-1.634199
1666311,2023-02-21 05:10:00.133,demo_model,1.0.1,25.258211,1.0,8.437500e+00,16.573775,-222.205785,44.0,0.279922
1666312,2023-02-21 05:28:14.204,demo_model,1.0.1,64.243332,0.0,3.181818e+00,13.291135,-261.132746,10230.0,-1.482410


In [12]:
import json
import pandas as pd

def extract_features(model_logs_df: pd.DataFrame) -> pd.DataFrame:
    json_str = "[" + (",".join(model_logs_df.features)) + "]"
    feature_df = pd.DataFrame(json.loads(json_str))
    feature_df = feature_df[sorted(feature_df.columns)]
    model_logs_df['occurred_at'] = model_logs_df['occurred_at'].apply(lambda x: x.replace(microsecond=0))
    model_logs_df['ds'] = model_logs_df['occurred_at'].apply(lambda x: x.strftime("%Y-%m-%d"))
    model_logs_df['hour'] = model_logs_df['occurred_at'].apply(lambda x: x.hour)
    return pd.concat([model_logs_df[['occurred_at', 'ds', 'hour', 'model_name', 'version', 'predictions']], feature_df], axis=1)

In [13]:
features_df = extract_features(demo_df)

In [14]:
features_df.head(5)

,occurred_at,ds,hour,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,2023-02-10 05:33:01,2023-02-10,5,demo_model,1.0.1,59.753181,0.0,1.904762e+00,-16.573775,-231.864749,10087.0,0.528394
1,2023-02-10 05:05:57,2023-02-10,5,demo_model,1.0.1,15.915874,0.0,9.375000e-01,-7.376024,-354.924267,44.0,-4.930488
2,2023-02-10 05:20:57,2023-02-10,5,demo_model,1.0.1,25.590763,1.0,5.909091e+00,-7.376024,-367.064925,495.0,-4.583076
3,2023-02-10 05:15:05,2023-02-10,5,demo_model,1.0.1,40.450287,0.0,5.000000e+08,-7.376024,-350.691669,3025.0,-4.816958
4,2023-02-10 05:36:09,2023-02-10,5,demo_model,1.0.1,24.397123,1.0,3.448276e-01,-7.376024,-356.517885,341.0,-4.216038


In [14]:
features_df.tail(5)

,occurred_at,ds,hour,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
1666309,2023-02-21 05:02:40,2023-02-21,5,demo_model,1.0.1,21.680845,1.0,7.692308,13.291135,-368.704926,561.0,-5.019625
1666310,2023-02-21 05:23:18,2023-02-21,5,demo_model,1.0.1,65.978500,1.0,2.647059,13.291135,-264.067607,10637.0,-1.634199
1666311,2023-02-21 05:10:00,2023-02-21,5,demo_model,1.0.1,25.258211,1.0,8.437500,16.573775,-222.205785,44.0,0.279922
1666312,2023-02-21 05:28:14,2023-02-21,5,demo_model,1.0.1,64.243332,0.0,3.181818,13.291135,-261.132746,10230.0,-1.482410
1666313,2023-02-21 05:12:56,2023-02-21,5,demo_model,1.0.1,19.288034,1.0,8.431373,16.573775,-222.481718,2321.0,0.279922


In [15]:
features_df.dtypes

occurred_at    datetime64[ns]
ds                     object
hour                    int64
model_name             object
version                object
predictions           float32
feature_1             float64
feature_2             float64
feature_3             float64
feature_4             float64
feature_5             float64
feature_6             float64
dtype: object

In [16]:
len(features_df.ds.unique())

88

In [17]:
features_df.hour.unique()

array([ 5,  1,  7, 11, 10, 18,  6, 20,  2, 21,  0, 23,  8,  3, 17, 22, 16,
       12, 15, 19,  4,  9, 13, 14])

In [18]:
features_df[(features_df['ds'] == '2023-02-10') & (features_df['hour'] == 5)]

,occurred_at,ds,hour,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,2023-02-10 05:33:01,2023-02-10,5,demo_model,1.0.1,59.753181,0.0,1.904762e+00,-16.573775,-231.864749,10087.0,0.528394
1,2023-02-10 05:05:57,2023-02-10,5,demo_model,1.0.1,15.915874,0.0,9.375000e-01,-7.376024,-354.924267,44.0,-4.930488
2,2023-02-10 05:20:57,2023-02-10,5,demo_model,1.0.1,25.590763,1.0,5.909091e+00,-7.376024,-367.064925,495.0,-4.583076
3,2023-02-10 05:15:05,2023-02-10,5,demo_model,1.0.1,40.450287,0.0,5.000000e+08,-7.376024,-350.691669,3025.0,-4.816958
4,2023-02-10 05:36:09,2023-02-10,5,demo_model,1.0.1,24.397123,1.0,3.448276e-01,-7.376024,-356.517885,341.0,-4.216038
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-02-10 05:35:44,2023-02-10,5,demo_model,1.0.1,17.575371,1.0,7.142857e-01,-16.573775,-223.450658,88.0,1.084640
996,2023-02-10 05:37:26,2023-02-10,5,demo_model,1.0.1,22.727821,1.0,4.761905e-01,-7.376024,-356.616071,1034.0,-4.216038
997,2023-02-10 05:12:26,2023-02-10,5,demo_model,1.0.1,24.809574,1.0,8.437500e+00,-7.376024,-367.860209,2519.0,-4.862648
998,2023-02-10 05:12:02,2023-02-10,5,demo_model,1.0.1,38.028652,0.0,0.000000e+00,-7.376024,-285.100373,4444.0,-1.694700


### Generate Whylogs Profiles

In [19]:
import json
import numpy as np

import whylogs as why
from whylogs import DatasetProfileView

In [20]:
feb_test_df = features_df[(features_df['ds'] == '2023-02-10') & (features_df['hour'] == 5)]

In [21]:
feb_test_df

,occurred_at,ds,hour,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,2023-02-10 05:33:01,2023-02-10,5,demo_model,1.0.1,59.753181,0.0,1.904762e+00,-16.573775,-231.864749,10087.0,0.528394
1,2023-02-10 05:05:57,2023-02-10,5,demo_model,1.0.1,15.915874,0.0,9.375000e-01,-7.376024,-354.924267,44.0,-4.930488
2,2023-02-10 05:20:57,2023-02-10,5,demo_model,1.0.1,25.590763,1.0,5.909091e+00,-7.376024,-367.064925,495.0,-4.583076
3,2023-02-10 05:15:05,2023-02-10,5,demo_model,1.0.1,40.450287,0.0,5.000000e+08,-7.376024,-350.691669,3025.0,-4.816958
4,2023-02-10 05:36:09,2023-02-10,5,demo_model,1.0.1,24.397123,1.0,3.448276e-01,-7.376024,-356.517885,341.0,-4.216038
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-02-10 05:35:44,2023-02-10,5,demo_model,1.0.1,17.575371,1.0,7.142857e-01,-16.573775,-223.450658,88.0,1.084640
996,2023-02-10 05:37:26,2023-02-10,5,demo_model,1.0.1,22.727821,1.0,4.761905e-01,-7.376024,-356.616071,1034.0,-4.216038
997,2023-02-10 05:12:26,2023-02-10,5,demo_model,1.0.1,24.809574,1.0,8.437500e+00,-7.376024,-367.860209,2519.0,-4.862648
998,2023-02-10 05:12:02,2023-02-10,5,demo_model,1.0.1,38.028652,0.0,0.000000e+00,-7.376024,-285.100373,4444.0,-1.694700


In [22]:
feb_whylogs_prof = why.log(feb_test_df[['feature_5', 'feature_6']]).view()

In [23]:
mar_test_df = features_df[(features_df['ds'] == '2023-03-10') & (features_df['hour'] == 5)]

In [24]:
mar_whylogs_prof = why.log(mar_test_df[['feature_5', 'feature_6']]).view()

In [25]:
feb_whylogs_prof.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,
feature_5,425.569275,420.134311,431.139744,0,1000,0,0,38148.000000,2184.743000,715.000000,0.000000,1000,11.000000,22.000000,33.000000,55.0000,3190.000000,6248.000000,9141.00000,14388.000000,3310.736242,SummaryType.COLUMN,0,1000,0,0,0,0
feature_6,233.000134,233.000000,233.011768,0,1000,0,0,1.785209,-1.798066,-1.543214,-6.845155,1000,-5.256787,-4.975246,-4.793975,-4.1657,0.373155,1.207462,1.48241,1.724903,2.350622,SummaryType.COLUMN,0,1000,0,0,0,0


### Visualize Whylogs Profiles

In [26]:
from whylogs.viz import NotebookProfileVisualizer

from whylogs.viz.utils.histogram_calculations import histogram_from_view
from whylogs.viz.utils.frequent_items_calculations import frequent_items_from_view

In [27]:
visualization = NotebookProfileVisualizer()
visualization.set_profiles(target_profile_view=feb_whylogs_prof, reference_profile_view=mar_whylogs_prof)

In [28]:
visualization.double_histogram(feature_name="feature_6")

### Serialize Whylogs Profiles

In [29]:
feb_whylogs_prof.serialize()[0:100]

b'WHY1\x00\xc2\x02\n\x0e \xa6\xfa\xc1\xcf\xfb0(\xa6\xfa\xc1\xcf\xfb0\x12\x10\n\tfeature_5\x12\x03\n\x01\x00\x12\x11\n\tfeature_6\x12\x04\n\x02\xa6P \xd4\x97\x01*\x14\x08\x05\x12\x10distribution/kll*\x11\x08\x08\x12\rtypes/bo'

### Generate Hourly Profiles using Fugue

In [30]:
import json
import pandas as pd

def profile_features(features_df: pd.DataFrame) -> pd.DataFrame:
    features_buf = why.log(features_df[['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6']]).view().serialize()
    predictions_buf = why.log(features_df[['predictions']]).view().serialize()
    profiled_features = features_df.head(1).copy()
    profiled_features = profiled_features.drop(['occurred_at'], axis=1)
    profiled_features = profiled_features.assign(features_profile=features_buf, predictions_profile = predictions_buf, sample_records=len(features_df))
    return profiled_features

In [31]:
feb_test_df.shape

(1000, 12)

In [32]:
profile_features(feb_test_df[(feb_test_df['ds'] == '2023-02-10') & (feb_test_df['hour'] == 5)])

,ds,hour,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,features_profile,predictions_profile,sample_records
0,2023-02-10,5,demo_model,1.0.1,59.753181,0.0,1.904762,-16.573775,-231.864749,10087.0,0.528394,b'WHY1\x00\x92\x03\n\x0e \xf1\xe5\xc2\xcf\xfb0(\xf1\xe5\xc2\xcf\xfb0\x12\x11\n\tfeature_2\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xe9\xe6\xc2\xcf\xfb0(\xe9\xe6\xc2\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000


In [33]:
from fugue import transform

hourly_feature_profile_df = transform(
    df=features_df, 
    using=profile_features, 
    schema="*-occurred_at+features_profile:binary,predictions_profile:binary,sample_records:long",
    partition=dict(by=['ds', 'hour', 'model_name', 'version']), 
    engine=None
)

In [34]:
hourly_feature_profile_df

,ds,hour,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,features_profile,predictions_profile,sample_records
0,2023-01-01,0,demo_model,1.0.1,28.874601,0.0,5.000000e+08,-13.291135,-303.546123,957.0,-6.954191,b'WHY1\x00\x92\x03\n\x0e \xda\xc0\xc3\xcf\xfb0(\xda\xc0\xc3\xcf\xfb0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xd8\xc1\xc3\xcf\xfb0(\xd8\xc1\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
1,2023-01-01,1,demo_model,1.0.1,34.759167,1.0,2.619048e+00,-13.291135,-226.094488,1111.0,-5.675494,b'WHY1\x00\x92\x03\n\x0e \x80\xc2\xc3\xcf\xfb0(\x80\xc2\xc3\xcf\xfb0\x12\x12\n\tfeature_3\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xe9\xc2\xc3\xcf\xfb0(\xe9\xc2\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
2,2023-01-01,2,demo_model,1.0.1,31.434237,0.0,5.000000e+08,-13.291135,-257.492325,1969.0,-3.698853,b'WHY1\x00\x92\x03\n\x0e \x8c\xc3\xc3\xcf\xfb0(\x8c\xc3\xc3\xcf\xfb0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xf4\xc3\xc3\xcf\xfb0(\xf4\xc3\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
3,2023-01-01,3,demo_model,1.0.1,26.973177,0.0,0.000000e+00,-13.291135,-260.120910,990.0,-3.830946,b'WHY1\x00\x92\x03\n\x0e \x97\xc4\xc3\xcf\xfb0(\x97\xc4\xc3\xcf\xfb0\x12\x12\n\tfeature_3\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \x83\xc5\xc3\xcf\xfb0(\x83\xc5\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
4,2023-01-01,4,demo_model,1.0.1,18.229908,0.0,2.590361e+00,-13.291135,-368.447875,55.0,-6.030039,b'WHY1\x00\x92\x03\n\x0e \xa6\xc5\xc3\xcf\xfb0(\xa6\xc5\xc3\xcf\xfb0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \x8e\xc6\xc3\xcf\xfb0(\x8e\xc6\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091,2023-03-29,3,demo_model,1.0.1,16.853651,0.0,1.739130e+00,16.573775,-226.308186,22.0,-0.404220,b'WHY1\x00\x8d\x03\n\x0e \x9d\x9f\xd4\xcf\xfb0(\x9d\x9f\xd4\xcf\xfb0\x12\x11\n\tfeature_4\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xe2\x9f\xd4\xcf\xfb0(\xe2\x9f\xd4\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,188
2092,2023-03-29,4,demo_model,1.0.1,30.805862,1.0,4.736842e+00,16.573775,-355.970607,44.0,-4.701136,b'WHY1\x00\x8d\x03\n\x0e \xff\x9f\xd4\xcf\xfb0(\xff\x9f\xd4\xcf\xfb0\x12\x11\n\tfeature_5\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xba\xa0\xd4\xcf\xfb0(\xba\xa0\xd4\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,125
2093,2023-03-29,5,demo_model,1.0.1,9.924586,1.0,3.730159e+00,16.573775,-356.026755,22.0,-2.173682,b'WHY1\x00\x8d\x03\n\x0e \xd4\xa0\xd4\xcf\xfb0(\xd4\xa0\xd4\xcf\xfb0\x12\x11\n\tfeature_5\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \x97\xa3\xd4\xcf\xfb0(\x97\xa3\xd4\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,67
2094,2023-03-29,6,demo_model,1.0.1,30.811672,0.0,2.500000e+00,7.376024,-264.412372,33.0,3.319832,b'WHY1\x00\x8d\x03\n\x0e \xb1\xa3\xd4\xcf\xfb0(\xb1\xa3\xd4\xcf\xfb0\x12\x11\n\tfeature_4\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xef\xa3\xd4\xcf\xfb0(\xef\xa3\xd4\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,74


### Merge Whylogs Profiles

In [36]:
type(feb_whylogs_prof)

whylogs.core.view.dataset_profile_view.DatasetProfileView

In [37]:
feb_whylogs_prof.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,
feature_5,425.569275,420.134311,431.139744,0,1000,0,0,38148.000000,2184.743000,715.000000,0.000000,1000,11.000000,22.000000,33.000000,55.0000,3190.000000,6248.000000,9141.00000,14388.000000,3310.736242,SummaryType.COLUMN,0,1000,0,0,0,0
feature_6,233.000134,233.000000,233.011768,0,1000,0,0,1.785209,-1.798066,-1.543214,-6.845155,1000,-5.256787,-4.975246,-4.793975,-4.1657,0.373155,1.207462,1.48241,1.724903,2.350622,SummaryType.COLUMN,0,1000,0,0,0,0


In [38]:
mar_whylogs_prof.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,
feature_5,13.000000,13.0,13.000649,0,267,0,0,154.000000,39.426966,33.000000,11.000000,267,11.000000,22.000000,22.000000,22.000000,44.000000,55.00000,110.000000,132.000000,24.119169,SummaryType.COLUMN,0,267,0,0,0,0
feature_6,134.000044,134.0,134.006735,0,267,0,0,1.815312,-1.352451,-0.373155,-5.019625,267,-5.019625,-4.793975,-4.630565,-3.987045,0.838042,1.48241,1.573573,1.785209,2.434174,SummaryType.COLUMN,0,267,0,0,0,0


In [39]:
merged_prof_view = feb_whylogs_prof.merge(mar_whylogs_prof)
merged_prof_view.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,
feature_5,425.569275,420.134311,431.139744,0,1267,0,0,38148.000000,1732.651934,220.000000,0.000000,1267,11.000000,22.000000,22.000000,44.000000,2310.000000,5500.000000,7546.000000,13640.000000,3068.471691,SummaryType.COLUMN,0,1267,0,0,0,0
feature_6,235.000137,235.000000,235.011870,0,1267,0,0,1.815312,-1.704160,-1.390994,-6.845155,1267,-5.235719,-4.952914,-4.770901,-4.140412,0.528394,1.268737,1.512827,1.724903,2.374470,SummaryType.COLUMN,0,1267,0,0,0,0


In [40]:
merge_test_df = features_df[((features_df['ds'] == '2023-02-10') | (features_df['ds'] == '2023-03-10')) & (features_df['hour'] == 5)]

In [41]:
merge_test_df['ds'].unique()

array(['2023-02-10', '2023-03-10'], dtype=object)

In [42]:
merged_whylogs_prof = why.log(merge_test_df[['feature_5', 'feature_6']]).view()

In [43]:
merged_whylogs_prof.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,
feature_5,425.569275,420.134311,431.139744,0,1267,0,0,38148.000000,1732.651934,220.000000,0.000000,1267,11.000000,22.000000,22.000000,44.000000,2365.000000,5577.000000,7590.000000,14234.000000,3068.471691,SummaryType.COLUMN,0,1267,0,0,0,0
feature_6,235.000137,235.000000,235.011870,0,1267,0,0,1.815312,-1.704160,-1.390994,-6.845155,1267,-5.235719,-4.952914,-4.770901,-4.140412,0.528394,1.268737,1.512827,1.724903,2.374470,SummaryType.COLUMN,0,1267,0,0,0,0


### Generate Daily Profiles

In [44]:
from functools import reduce

def profile_reduce(hourly_profiles_df: pd.DataFrame) -> pd.DataFrame:
    features_buf = reduce(
        lambda acc, x: acc.merge(x),
        hourly_profiles_df.features_profile.apply(DatasetProfileView.deserialize),
    ).serialize()
    predictions_buf = reduce(
        lambda acc, x: acc.merge(x),
        hourly_profiles_df.predictions_profile.apply(DatasetProfileView.deserialize),
    ).serialize()
    records = hourly_profiles_df.sample_records.sum()
    daily_profiles_df = hourly_profiles_df.head(1).copy()
    daily_profiles_df = daily_profiles_df.drop(['hour'], axis=1)
    daily_profiles_df = daily_profiles_df.assign(features_profile=features_buf, predictions_profile = predictions_buf, sample_records=records)
    return daily_profiles_df

In [45]:
hourly_feature_profile_df

,ds,hour,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,features_profile,predictions_profile,sample_records
0,2023-01-01,0,demo_model,1.0.1,28.874601,0.0,5.000000e+08,-13.291135,-303.546123,957.0,-6.954191,b'WHY1\x00\x92\x03\n\x0e \xda\xc0\xc3\xcf\xfb0(\xda\xc0\xc3\xcf\xfb0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xd8\xc1\xc3\xcf\xfb0(\xd8\xc1\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
1,2023-01-01,1,demo_model,1.0.1,34.759167,1.0,2.619048e+00,-13.291135,-226.094488,1111.0,-5.675494,b'WHY1\x00\x92\x03\n\x0e \x80\xc2\xc3\xcf\xfb0(\x80\xc2\xc3\xcf\xfb0\x12\x12\n\tfeature_3\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xe9\xc2\xc3\xcf\xfb0(\xe9\xc2\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
2,2023-01-01,2,demo_model,1.0.1,31.434237,0.0,5.000000e+08,-13.291135,-257.492325,1969.0,-3.698853,b'WHY1\x00\x92\x03\n\x0e \x8c\xc3\xc3\xcf\xfb0(\x8c\xc3\xc3\xcf\xfb0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xf4\xc3\xc3\xcf\xfb0(\xf4\xc3\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
3,2023-01-01,3,demo_model,1.0.1,26.973177,0.0,0.000000e+00,-13.291135,-260.120910,990.0,-3.830946,b'WHY1\x00\x92\x03\n\x0e \x97\xc4\xc3\xcf\xfb0(\x97\xc4\xc3\xcf\xfb0\x12\x12\n\tfeature_3\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \x83\xc5\xc3\xcf\xfb0(\x83\xc5\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
4,2023-01-01,4,demo_model,1.0.1,18.229908,0.0,2.590361e+00,-13.291135,-368.447875,55.0,-6.030039,b'WHY1\x00\x92\x03\n\x0e \xa6\xc5\xc3\xcf\xfb0(\xa6\xc5\xc3\xcf\xfb0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \x8e\xc6\xc3\xcf\xfb0(\x8e\xc6\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091,2023-03-29,3,demo_model,1.0.1,16.853651,0.0,1.739130e+00,16.573775,-226.308186,22.0,-0.404220,b'WHY1\x00\x8d\x03\n\x0e \x9d\x9f\xd4\xcf\xfb0(\x9d\x9f\xd4\xcf\xfb0\x12\x11\n\tfeature_4\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xe2\x9f\xd4\xcf\xfb0(\xe2\x9f\xd4\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,188
2092,2023-03-29,4,demo_model,1.0.1,30.805862,1.0,4.736842e+00,16.573775,-355.970607,44.0,-4.701136,b'WHY1\x00\x8d\x03\n\x0e \xff\x9f\xd4\xcf\xfb0(\xff\x9f\xd4\xcf\xfb0\x12\x11\n\tfeature_5\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xba\xa0\xd4\xcf\xfb0(\xba\xa0\xd4\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,125
2093,2023-03-29,5,demo_model,1.0.1,9.924586,1.0,3.730159e+00,16.573775,-356.026755,22.0,-2.173682,b'WHY1\x00\x8d\x03\n\x0e \xd4\xa0\xd4\xcf\xfb0(\xd4\xa0\xd4\xcf\xfb0\x12\x11\n\tfeature_5\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \x97\xa3\xd4\xcf\xfb0(\x97\xa3\xd4\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,67
2094,2023-03-29,6,demo_model,1.0.1,30.811672,0.0,2.500000e+00,7.376024,-264.412372,33.0,3.319832,b'WHY1\x00\x8d\x03\n\x0e \xb1\xa3\xd4\xcf\xfb0(\xb1\xa3\xd4\xcf\xfb0\x12\x11\n\tfeature_4\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xef\xa3\xd4\xcf\xfb0(\xef\xa3\xd4\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,74


In [46]:
profile_reduce(hourly_feature_profile_df[hourly_feature_profile_df['ds'] == '2023-01-01'])

,ds,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,features_profile,predictions_profile,sample_records
0,2023-01-01,demo_model,1.0.1,28.874601,0.0,500000000.0,-13.291135,-303.546123,957.0,-6.954191,b'WHY1\x00\x93\x03\n\x0e \xda\xc0\xc3\xcf\xfb0(\xda\xc0\xc3\xcf\xfb0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xd8\xc1\xc3\xcf\xfb0(\xd8\xc1\xc3\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,24001


In [47]:
from fugue import transform

daily_feature_profile_df = transform(
    df=hourly_feature_profile_df, 
    using=profile_reduce, 
    schema="*-hour",
    partition=dict(by=['ds', 'model_name', 'version']), 
    engine=None
)

In [48]:
daily_feature_profile_df

,ds,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,features_profile,predictions_profile,sample_records
0,2023-01-01,demo_model,1.0.1,28.874601,0.0,5.000000e+08,-13.291135,-303.546123,957.0,-6.954191,b'WHY1\x00\x93\x03\n\x0e \xc7\xfa\xcf\x8b\xfa0(\xc7\xfa\xcf\x8b\xfa0\x12\x12\n\tfeature_4\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xc6\xfb\xcf\x8b\xfa0(\xc6\xfb\xcf\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,24001
1,2023-01-02,demo_model,1.0.1,17.241968,1.0,3.750000e+00,-0.000000,-226.315937,44.0,-6.295416,b'WHY1\x00\x93\x03\n\x0e \xe4\x94\xd0\x8b\xfa0(\xe4\x94\xd0\x8b\xfa0\x12\x10\n\tfeature_1\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xd9\x95\xd0\x8b\xfa0(\xd9\x95\xd0\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,23999
2,2023-01-03,demo_model,1.0.1,5.838543,1.0,5.000000e+08,13.291135,-222.223722,33.0,-6.190258,b'WHY1\x00\x93\x03\n\x0e \xe2\xae\xd0\x8b\xfa0(\xe2\xae\xd0\x8b\xfa0\x12\x10\n\tfeature_1\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xc7\xaf\xd0\x8b\xfa0(\xc7\xaf\xd0\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,24000
3,2023-01-04,demo_model,1.0.1,15.285272,1.0,2.631579e+00,16.573775,-355.747054,363.0,-6.324387,b'WHY1\x00\x93\x03\n\x0e \xc4\xca\xd0\x8b\xfa0(\xc4\xca\xd0\x8b\xfa0\x12\x12\n\tfeature_2\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xb0\xcb\xd0\x8b\xfa0(\xb0\xcb\xd0\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,24000
4,2023-01-05,demo_model,1.0.1,5.200397,1.0,5.000000e+08,7.376024,-223.297205,33.0,-6.862306,b'WHY1\x00\x93\x03\n\x0e \x87\xe4\xd0\x8b\xfa0(\x87\xe4\xd0\x8b\xfa0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xed\xe4\xd0\x8b\xfa0(\xed\xe4\xd0\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,24000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2023-03-25,demo_model,1.0.1,20.648773,0.0,8.088235e-01,-16.573775,-368.562824,22.0,-7.097175,b'WHY1\x00\x92\x03\n\x0e \xde\xf7\xdf\x8b\xfa0(\xde\xf7\xdf\x8b\xfa0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \x99\xf8\xdf\x8b\xfa0(\x99\xf8\xdf\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,4248
84,2023-03-26,demo_model,1.0.1,13.163588,1.0,2.500000e+00,-13.291135,-263.689394,33.0,-6.689304,b'WHY1\x00\x93\x03\n\x0e \xb8\x88\xe0\x8b\xfa0(\xb8\x88\xe0\x8b\xfa0\x12\x12\n\tfeature_6\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xf4\x88\xe0\x8b\xfa0(\xf4\x88\xe0\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,3784
85,2023-03-27,demo_model,1.0.1,29.717026,1.0,2.500000e+00,-0.000000,-213.566814,110.0,-5.962720,b'WHY1\x00\x93\x03\n\x0e \x8d\x98\xe0\x8b\xfa0(\x8d\x98\xe0\x8b\xfa0\x12\x12\n\tfeature_2\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xca\x98\xe0\x8b\xfa0(\xca\x98\xe0\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,3686
86,2023-03-28,demo_model,1.0.1,11.663711,1.0,5.520833e+00,13.291135,-263.896567,44.0,-6.435433,b'WHY1\x00\x92\x03\n\x0e \x82\xa9\xe0\x8b\xfa0(\x82\xa9\xe0\x8b\xfa0\x12\x10\n\tfeature_1\x12\x0...,b'WHY1\x00\xb1\x02\n\x0e \xbb\xa9\xe0\x8b\xfa0(\xbb\xa9\xe0\x8b\xfa0\x12\x12\n\x0bpredictions\x1...,5290


### Scaling up with Fugue & Dask

#### DASK

In [48]:
from fugue import transform

hourly_feature_profile_df = transform(
    df=features_df, 
    using=profile_features, 
    schema="*-occurred_at+features_profile:binary,predictions_profile:binary,sample_records:long",
    partition=dict(by=['ds', 'hour', 'model_name', 'version']), 
    engine="dask"
)

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [49]:
hourly_feature_profile_df.head(5)

/opt/conda/lib/python3.10/site-packages/distributed/client.py:3095: UserWarning: Sending large graph of size 125.58 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


,ds,hour,model_name,version,predictions,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,features_profile,predictions_profile,sample_records
0,2023-01-01,2,demo_model,1.0.1,31.434237,0.0,5.000000e+08,-13.291135,-257.492325,1969.0,-3.698853,b'WHY1\x00\x92\x03\n\x0e \xa9\xe8\xdd\xcf\xfb0(\xa9\xe8\xdd\xcf\xfb0\x12\x12\n\tfeature_4\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xbd\xea\xdd\xcf\xfb0(\xbd\xea\xdd\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
1,2023-01-02,9,demo_model,1.0.1,19.137638,1.0,1.400000e+01,13.291135,-280.842539,44.0,6.013381,b'WHY1\x00\x92\x03\n\x0e \x94\xeb\xdd\xcf\xfb0(\x94\xeb\xdd\xcf\xfb0\x12\x12\n\tfeature_4\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xf8\xec\xdd\xcf\xfb0(\xf8\xec\xdd\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
2,2023-01-02,22,demo_model,1.0.1,11.617042,1.0,4.473684e+00,13.291135,-336.947768,110.0,-6.111603,b'WHY1\x00\x92\x03\n\x0e \xd7\xed\xdd\xcf\xfb0(\xd7\xed\xdd\xcf\xfb0\x12\x12\n\tfeature_4\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \x9b\xef\xdd\xcf\xfb0(\x9b\xef\xdd\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
3,2023-01-03,2,demo_model,1.0.1,34.734276,1.0,5.000000e+08,13.291135,-222.465304,1584.0,-4.064077,b'WHY1\x00\x92\x03\n\x0e \x87\xf0\xdd\xcf\xfb0(\x87\xf0\xdd\xcf\xfb0\x12\x11\n\tfeature_2\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xde\xf1\xdd\xcf\xfb0(\xde\xf1\xdd\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,1000
4,2023-01-04,10,demo_model,1.0.1,6.508048,0.0,5.000000e+00,7.376024,-352.856626,44.0,5.019625,b'WHY1\x00\x92\x03\n\x0e \x98\xf2\xdd\xcf\xfb0(\x98\xf2\xdd\xcf\xfb0\x12\x11\n\tfeature_2\x12\x0...,b'WHY1\x00\xb0\x02\n\x0e \xfc\xf3\xdd\xcf\xfb0(\xfc\xf3\xdd\xcf\xfb0\x12\x12\n\x0bpredictions\x1...,999


Simlarly, we can also use `engine="ray"` `engine="spark"` as the backend engines to scale it up seamlessly with `Ray` or `Spark`.